In [3]:
import Pkg 
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("NLS_Solver")
Pkg.add("ForwardDiff")
Pkg.add("JSON")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.9/Project.toml`
  [682c06a0] + JSON v0.21.4
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [64]:
using NLS_Solver

domain = range(1.0, 10.0, length=100)
image = map(x -> 1/x + 2 + 3 * x, domain) 

A = [domain image]

n_params = 3 
n_samples = size(A)[1]

function model(x::Real, params::AbstractVector)
    return params[1] * 1/x + params[2] + params[3] * x 
end

function residue(X::AbstractVector, Y::AbstractVector, params::AbstractVector)
        map(zip(X, Y)) do (x, y) 
            (model(x, params) - y) 
        end
end

nls = create_NLS_problem_using_ForwardDiff(params -> residue(A[:,1], A[:, 2], params), n_params => n_samples)

result = solve(nls, [1.0, 1.0, 1.0], LevenbergMarquardt_Conf())

LevenbergMarquardt_Result{Float64}(true, 8, 7.284737645639428e-21, [1.0000000001137384, 1.999999999928923, 3.0000000000074656])

In [23]:
using CSV, DataFrames

# Corresponds to query https://dune.com/queries/3087337
# Profuces DataFrame with columns ["volume", "fee", "mean_fee"]
function dex_fee_and_volume(path)
    token_pairs = []
    for file in readdir(path)
        filename = split(file, ".")[1]
        filepath = joinpath(path, file)
        df = CSV.read(filepath, DataFrame, types=[String, Float64, Float64], silencewarnings=true)

        df = df[.!ismissing.(df.fee), :]
        df.fee = abs.(df.fee)
        df.mean_fee = df.fee ./ df.dollar_value_sold
        df.volume = copy(df.dollar_value_sold)
        select!(df, Not(:dollar_value_sold))

        df = sort(df, :volume, rev=true)
        push!(token_pairs, (filename, df))
        
    end

    return Dict(token_pairs)
end 

dex_fee_and_volume_df = dex_fee_and_volume("../queries/dex_fee_and_volume")

using Plots

for token_pair in keys(dex_fee_and_volume_df)
    token_pair_df = dex_fee_and_volume_df[token_pair][1:6,:]
    plot = bar(token_pair_df.market, token_pair_df.volume,  yaxis="Volume", xaxis="Market", size=(800, 400), legend=false)
    plot!(twinx(), token_pair_df.market, token_pair_df.mean_fee,  yaxis="Mean Fee",  size=(800, 400), color=:red, legend=false)
    savefig(joinpath("..", "output", "dex_fee_and_volume", "plot_" * token_pair * ".png"))
end


In [24]:
using CSV, DataFrames
# Corresponds to queries under the category https://dune.com/queries/3106546
# Produces DataFrame with columns ["token_pair", "total_volume"]
function dex_market_shares(path)
    ranking = []
    for file in readdir(path)
        filename = split(file, ".")[1]
        filepath = joinpath(path, file)
        df = CSV.read(filepath, DataFrame, types=[String, Float64], silencewarnings=true)

        df = sort(df, Not(Symbol(filename)), rev=true) # Sort by the numerical column
        push!(ranking, (filename, df))
        
    end
    return Dict(ranking)
end 

dex_market_shares_df = Dict(
    "fees"=> dex_market_shares("../queries/market_share/fees"), 
    "volume"=> dex_market_shares("../queries/market_share/volume")
)

using Plots


for dependent_var in keys(dex_market_shares_df)
    for category in keys(dex_market_shares_df[dependent_var])
        category_df = dex_market_shares_df[dependent_var][category]
        # remove negative entries
        category_df = category_df[category_df[!,2] .> 0, :]
        plot = pie(category_df[1:10,1], category_df[1:10,2], title=dependent_var * " by " * category, legend=true)
        savefig(joinpath("..", "output", "market_share", dependent_var, "plot_" * category * ".png"))
    end
end

In [25]:
using CSV, DataFrames

# Corresponds to query https://dune.com/queries/3106936
# Produces DataFrame with columns ["amount_usd", "mean_fee"]
function trades_amountusd_fee(path)
    token_venue = []
    for file in readdir(path)
        filename = split(file, ".")[1]
        filepath = joinpath(path, file)
        df = CSV.read(filepath, DataFrame, types=[Float64, Float64], silencewarnings=true)

        df = df[.!ismissing.(df.fee), :]
        df.mean_fee = df.fee ./ df.dollar_value_sold
        select!(df, Not(:fee))

        push!(token_venue, (filename, df))
    end

    return Dict(token_venue)
end 

trades_amountusd_fee_df = trades_amountusd_fee("../queries/trades/amountusd_fee")

using Plots, NLS_Solver

for token_venue in keys(trades_amountusd_fee_df)
    token_pair, venue = split(token_venue, "_")
    token_venue_df = trades_amountusd_fee_df[token_venue]
    plot = scatter(token_venue_df.amount_usd, token_venue_df.mean_fee, title=token_pair * " on " * venue, legend=false)
end

LoadError: ArgumentError: column name :fee not found in the data frame

In [24]:
using JSON, DataFrames
using Base.MathConstants

function sqrtPriceX96ToFloat(sqrtPriceX96::BigInt, decimalsToken0::Int)
    numerator1 = sqrtPriceX96 * sqrtPriceX96
    numerator2 = BigInt(10) ^ decimalsToken0
    return numerator1 * numerator2 / (BigInt(2) ^ 192)
end
# Corresponds to Graph Protocol query fetch_uniswapv3_pool
# Produces DataFrame with columns ["amount_usd", "mean_fee"]
function graph_uniswapv3_pools(path)
    token_venue = []
    for file in readdir(path)
        filename = split(file, ".")[1]
        filepath = joinpath(path, file)
        
        json = JSON.parsefile(filepath)
        df = DataFrame(timestamp = Int64[], amount_usd = Float32[], fee_tier = Int64[], price = Float64[])
        for entry in json 
            push!(df, Dict(:timestamp => parse(Int, entry["timestamp"]), :amount_usd => parse(Float32, entry["amountUSD"]), :fee_tier => parse(Int, entry["pool"]["feeTier"]), :price => sqrtPriceX96ToFloat(parse(BigInt, entry["sqrtPriceX96"]), 18)))
        end

        push!(token_venue, (filename, df))
    end

    return Dict(token_venue)
end 

graph_uniswapv3_pools_df = graph_uniswapv3_pools("../queries/graph_protocol/uniswapv3_pool_fees")

using Plots

for token_pair in keys(graph_uniswapv3_pools_df)
    df = graph_uniswapv3_pools_df[token_pair]
    # Aggregate timestamp by day
    df.day = floor.(Int64, df.timestamp ./ (24 * 60 * 60))
    df = combine(groupby(df, [:day, :fee_tier]), :amount_usd => sum)
    # plot timeseries with area chart colored by fee tier
    areaplot(df.day, df.amount_usd_sum, group=df.fee_tier, title=token_pair, legend=:topleft)
    savefig(joinpath("..", "output", "graph_protocol", "uniswapv3_pool_fees", "plot_" * token_pair * ".png"))
end

df[1:5, :] = 5×3 DataFrame
 Row │ day    fee_tier  amount_usd_sum
     │ Int64  Int64     Float32
─────┼─────────────────────────────────
   1 │ 19632       500       1.32541e7
   2 │ 19645       500       3.62242e6
   3 │ 19657       500       9.05356e6
   4 │ 19650       500       5.72713e6
   5 │ 19654       500       1.85076e7
df[1:5, :] = 5×3 DataFrame
 Row │ day    fee_tier  amount_usd_sum
     │ Int64  Int64     Float32
─────┼─────────────────────────────────
   1 │ 19597       500       4.96278e5
   2 │ 19290      3000       3.5117e5
   3 │ 19368       500       6.66303e5
   4 │ 19272       500       4.29847e5
   5 │ 19292       500       8.03799e5
